In [ ]:
import numpy as np
import pandas as pd
import sys
import os
import shutil
from sklearn.metrics import silhouette_score

<h2>데이터 전처리<h2>

In [ ]:
def read_text_files_to_dataframe(folder_path):
    
    latitude = []
    longitude = []
    date = []
    time = []

    # 폴더 내의 모든 텍스트 파일을 읽음
    for filename in os.listdir(folder_path):
        if filename.endswith(".txt"):
            file_path = os.path.join(folder_path, filename)
            
            # 텍스트 파일을 DataFrame에 추가

            with open(file_path, 'r') as file:
                data = file.read()

            data = data.split('\n')[6:-1]
            for i in range(len(data)):
                line = data[i].split(',')
                latitude.append(line[0])
                longitude.append(line[1])
                date.append(line[5])
                time.append(line[6])

    df = pd.DataFrame({"latitude":latitude, "longitude":longitude, "date":date, "time":time})
    df['latitude'] = df['latitude'].astype(float)
    df['longitude'] = df['longitude'].astype(float)
    df['datetime'] = pd.to_datetime(df['date'] + ' ' + df['time'], format='%Y-%m-%d %H:%M:%S')
    df['datetime'] = df['datetime'].dt.floor('T')
    df = df.drop(['date', 'time'], axis=1)
    df = df.drop_duplicates(['datetime'], ignore_index=True)

    return df

In [ ]:
folder_path = "C:\\Users\\sk002\\OneDrive\\바탕 화면\\학교\\Yoodori\\Geolife Trajectories 1.3\\Data\\000\\Trajectory"

dataframe = read_text_files_to_dataframe(folder_path)

dataframe

<h2>의미장소 추출<h2>

In [ ]:
from pyclustering.cluster.gmeans import gmeans
from pyclustering.cluster import cluster_visualizer
from pyclustering.cluster import cluster_visualizer_multidim
import folium

In [ ]:
def gmeans_(df):
    # 두 열을 선택하고 넘파이 배열로 변환
    selected_columns = ['latitude', 'longitude']
    result_list = df[selected_columns].values.tolist()
    
    gmeans_instance = gmeans(result_list).process()

    # Extract clustering results: clusters and their centers
    clusters = gmeans_instance.get_clusters()
    centers = gmeans_instance.get_centers()

    return clusters, centers
    

In [ ]:
def map(centers, i):

    # 예제 데이터 생성 (서울의 위도, 경도)
    locations = centers

    # 지도의 중심 좌표 설정
    map_center = [sum(x[0] for x in locations) / len(locations), sum(x[1] for x in locations) / len(locations)]

    # Folium 맵 생성
    mymap = folium.Map(location=map_center, zoom_start=12)

    # 위치 데이터를 지도에 마커로 표시
    for loc in locations:
        folium.Marker(location=loc, popup='Location').add_to(mymap)

    # 지도를 HTML 파일로 저장하거나 주피터 노트북에서 바로 표시
    file_name = f'map{i}.html'
    mymap.save(file_name)


In [ ]:
def fileReader(folder_path, filename):
    latitude = []
    longitude = []
    date = []
    time = []
    file_path = os.path.join(folder_path, filename)
            
    # 텍스트 파일을 DataFrame에 추가

    with open(file_path, 'r') as file: 
        data = file.read()

    data = data.split('\n')[6:-1]
    for i in range(len(data)):
        line = data[i].split(',')
        latitude.append(line[0])
        longitude.append(line[1])
        date.append(line[5])
        time.append(line[6])

    df = pd.DataFrame({"latitude":latitude, "longitude":longitude, "date":date, "time":time})

    return df

In [ ]:
def app(folder_path):
    
    j = 0
    data_dict = {}
    # 폴더 내의 모든 텍스트 파일을 읽음
    for filename in os.listdir(folder_path):

        df = fileReader(folder_path, filename)

        df['latitude'] = df['latitude'].astype(float)
        df['longitude'] = df['longitude'].astype(float)
        df['datetime'] = pd.to_datetime(df['date'] + ' ' + df['time'], format='%Y-%m-%d %H:%M:%S')
        df['datetime'] = df['datetime'].dt.floor('T')
        df = df.drop(['date', 'time'], axis=1)
        df = df.drop_duplicates(['datetime'], ignore_index=True)

        clusters, centers = gmeans(df)

        data_dict[j] = pd.DataFrame({"clusters":clusters, "centers":centers})
        
        for k in range(len(data_dict[j].clusters)):
            if (len(data_dict[j].clusters[k]) < 6):
                data_dict[j].drop(index=k, inplace=True)
        data_dict[j] = data_dict[j].sort_index(axis=1)

        map(centers, j)
        
        j += 1
    return data_dict

In [ ]:
#folder_name = r'C:\Users\sk002\OneDrive\바탕 화면\학교\Yoodori\Geolife Trajectories 1.3\Data\000\Trajectory'
#data_dict = app(folder_name)

In [ ]:
def fileReader(filename):

    latitude = []   # 위도
    longitude = []  # 경도
    date = []       # 날짜
    time = []       # 시간

    with open(filename, 'r') as file:
        data = file.read()

    # 데이터에 불필요한 부분 제거
    # 추후 데이터 형식에 따라 수정 필요 *
    data = data.split('\n')[6:-1]
    for i in range(len(data)):
        line = data[i].split(',')
        latitude.append(line[0])    # 위도
        longitude.append(line[1])   # 경도
        date.append(line[5])        # 날짜
        time.append(line[6])        # 시간

    df = pd.DataFrame({"latitude":latitude, "longitude":longitude, "date":date, "time":time})

    return df

In [ ]:
def gmeansFit(df):
    # 두 열을 선택하고 넘파이 배열로 변환
    selectedColumns = ['latitude', 'longitude']
    resultList = df[selectedColumns].values.tolist()    # 리스트로 변환
    
    gmeansInstance = gmeans(resultList).process()       # 클러스터링

    centers = gmeansInstance.get_centers()              # 클러스터의 중심 (의미장소)
    clusters = gmeansInstance.get_clusters()            # 분류된 클러스터


    return clusters, centers

In [ ]:
def gmeansFunc(file_name):
    
    j = 0
    data_dict = {}

    df = fileReader(file_name)

    # 위도 경도 데이터 형식 변경
    df['latitude'] = df['latitude'].astype(float)
    df['longitude'] = df['longitude'].astype(float)
    # 날짜, 시간 데이터 병합
    df['datetime'] = pd.to_datetime(df['date'] + ' ' + df['time'], format='%Y-%m-%d %H:%M:%S')
    df['datetime'] = df['datetime'].dt.floor('T')
    df = df.drop(['date', 'time'], axis=1)
    # 1분 단위로 데이터 병합
    df = df.drop_duplicates(['datetime'], ignore_index=True)

    # 의미장소 추출
    clusters, centers = gmeansFit(df)
        
    # 딕셔너리 형식으로 데이터 저장
    data_dict[j] = pd.DataFrame({"clusters":clusters, "centers":centers})
        
    # 클러스터가 10개 미만인 의미장소 제거
    for k in range(len(data_dict[j].clusters)):
        if (len(data_dict[j].clusters[k]) < 10):
            data_dict[j].drop(index=k, inplace=True)
    data_dict[j] = data_dict[j].sort_index(axis=1)
        
    j += 1
    return data_dict

In [ ]:
file_name = r'C:\Users\sk002\OneDrive\바탕 화면\학교\Yoodori\Geolife Trajectories 1.3\Data\000\Trajectory\20090403011657.txt'
data_dict = gmeansFunc(file_name)

<h2>서버에 올릴 코드<h2>

In [1]:
from pyclustering.cluster.gmeans import gmeans
from pyclustering.cluster import cluster_visualizer
from pyclustering.cluster import cluster_visualizer_multidim
import numpy as np
import pandas as pd
import os
import folium

In [39]:
def fileReader(filename):

    latitude = []   # 위도
    longitude = []  # 경도
    date = []       # 날짜
    time = []       # 시간

    with open(filename, 'r') as file:
        data = file.read()

    # 데이터에 불필요한 부분 제거
    # 추후 데이터 형식에 따라 수정 필요 *
    data = data.split('\n')[6:-1]
    for i in range(len(data)):
        line = data[i].split(',')
        latitude.append(line[0])    # 위도
        longitude.append(line[1])   # 경도
        date.append(line[5])        # 날짜
        time.append(line[6])        # 시간

    df = pd.DataFrame({"latitude":latitude, "longitude":longitude, "date":date, "time":time})

    
    df['latitude'] = df['latitude'].astype(float)
    df['longitude'] = df['longitude'].astype(float)
    df['datetime'] = pd.to_datetime(df['date'] + ' ' + df['time'], format='%Y-%m-%d %H:%M:%S')
    df['datetime'] = df['datetime'].dt.floor('T')
    # 시간대와 요일 추가
    # 시간대 형식 : f00t04 f20t24
    # 4시간 단위로 분리
    df['hour_block'] = 'f' + ((df['datetime'].dt.hour) // 4 * 4).astype(str).str.zfill(2) + 't' + ((df['datetime'].dt.hour + 4) // 4 * 4).astype(str).str.zfill(2)
    df['day_of_week'] = df['datetime'].dt.day_name()
    print(df)
    df = df.drop(['date', 'time'], axis=1)
    df = df.drop_duplicates(['datetime'], ignore_index=True)

    return df

In [3]:
# 의미장소 추출
def gmeansFit(df):
    # 두 열을 선택하고 넘파이 배열로 변환
    selectedColumns = ['latitude', 'longitude']
    resultList = df[selectedColumns].values.tolist()    # 리스트로 변환
    
    gmeansInstance = gmeans(resultList).process()       # 클러스터링

    centers = gmeansInstance.get_centers()              # 클러스터의 중심 (의미장소)
    clusters = gmeansInstance.get_clusters()            # 분류된 클러스터

 

    return clusters, centers

In [4]:
def gmeansFunc(df):


    clusters, centers = gmeansFit(df)

    data_df = pd.DataFrame({"clusters":clusters, "centers":centers})
        
    for k in range(len(data_df.clusters)):
        if (len(data_df.clusters[k]) < 10):
            data_df.drop(index=k, inplace=True)
    data_df = data_df.sort_index(axis=1)
    data_df = data_df.reset_index(drop=True)
    
    # 의미장소에 요일, 시간 데이터를 추가
    # 제일 많은 시간대, 요일을 의미장소에 추가
    

    return data_df

In [40]:
file_name = r'C:\Users\sk002\OneDrive\바탕 화면\학교\Yoodori\Geolife Trajectories 1.3\Data\003\Trajectory\20090403011657.txt'
df1 = fileReader(file_name)
data_df_1 = gmeansFunc(df1)

        latitude   longitude        date      time            datetime   
0      40.001520  116.311859  2009-04-03  01:16:57 2009-04-03 01:16:00  \
1      40.001437  116.312250  2009-04-03  01:17:02 2009-04-03 01:17:00   
2      40.001434  116.312406  2009-04-03  01:17:07 2009-04-03 01:17:00   
3      40.001418  116.312536  2009-04-03  01:17:12 2009-04-03 01:17:00   
4      40.001398  116.312649  2009-04-03  01:17:17 2009-04-03 01:17:00   
...          ...         ...         ...       ...                 ...   
14181  40.009646  116.315193  2009-04-04  14:34:49 2009-04-04 14:34:00   
14182  40.009631  116.315176  2009-04-04  14:34:54 2009-04-04 14:34:00   
14183  40.009592  116.315135  2009-04-04  14:34:59 2009-04-04 14:34:00   
14184  40.009575  116.315145  2009-04-04  14:35:04 2009-04-04 14:35:00   
14185  40.009599  116.315182  2009-04-04  14:35:09 2009-04-04 14:35:00   

      hour_block day_of_week  
0         f00t04      Friday  
1         f00t04      Friday  
2         f00t04  

In [41]:
file_name = r'C:\Users\sk002\OneDrive\바탕 화면\학교\Yoodori\Geolife Trajectories 1.3\Data\003\Trajectory\20090120002837.txt'
df2 = fileReader(file_name)
data_df_2 = gmeansFunc(df2)

        latitude   longitude        date      time            datetime   
0      39.999840  116.327191  2009-01-20  00:28:37 2009-01-20 00:28:00  \
1      39.999873  116.327182  2009-01-20  00:28:42 2009-01-20 00:28:00   
2      39.999899  116.327115  2009-01-20  00:28:47 2009-01-20 00:28:00   
3      39.999933  116.326987  2009-01-20  00:28:52 2009-01-20 00:28:00   
4      39.999953  116.326813  2009-01-20  00:28:57 2009-01-20 00:28:00   
...          ...         ...         ...       ...                 ...   
11075  32.055799  121.271346  2009-01-21  02:19:25 2009-01-21 02:19:00   
11076  32.055712  121.271453  2009-01-21  02:19:30 2009-01-21 02:19:00   
11077  32.055684  121.271479  2009-01-21  02:19:35 2009-01-21 02:19:00   
11078  32.055660  121.271520  2009-01-21  02:19:40 2009-01-21 02:19:00   
11079  32.055686  121.271484  2009-01-21  02:19:45 2009-01-21 02:19:00   

      hour_block day_of_week  
0         f00t04     Tuesday  
1         f00t04     Tuesday  
2         f00t04  

In [42]:
file_name = r'C:\Users\sk002\OneDrive\바탕 화면\학교\Yoodori\Geolife Trajectories 1.3\Data\003\Trajectory\20090221034838.txt'
df3 = fileReader(file_name)
data_df_3 = gmeansFunc(df3)

       latitude   longitude        date      time            datetime   
0     39.999168  116.334129  2009-02-21  03:48:38 2009-02-21 03:48:00  \
1     39.999200  116.334175  2009-02-21  03:48:43 2009-02-21 03:48:00   
2     39.999224  116.334233  2009-02-21  03:48:48 2009-02-21 03:48:00   
3     39.999264  116.334250  2009-02-21  03:48:53 2009-02-21 03:48:00   
4     39.999310  116.334270  2009-02-21  03:48:58 2009-02-21 03:48:00   
...         ...         ...         ...       ...                 ...   
8492  39.997613  116.338555  2009-02-21  22:56:36 2009-02-21 22:56:00   
8493  39.997581  116.338660  2009-02-21  22:56:41 2009-02-21 22:56:00   
8494  39.997600  116.338697  2009-02-21  22:56:46 2009-02-21 22:56:00   
8495  39.997600  116.338697  2009-02-21  22:56:49 2009-02-21 22:56:00   
8496  39.997596  116.338699  2009-02-21  22:56:51 2009-02-21 22:56:00   

     hour_block day_of_week  
0        f00t04    Saturday  
1        f00t04    Saturday  
2        f00t04    Saturday  
3  

In [43]:
file_name = r'C:\Users\sk002\OneDrive\바탕 화면\학교\Yoodori\Geolife Trajectories 1.3\Data\003\Trajectory\20090214045230.txt'
df4 = fileReader(file_name)
data_df_4 = gmeansFunc(df4)

       latitude   longitude        date      time            datetime   
0     39.998569  116.339200  2009-02-14  04:52:30 2009-02-14 04:52:00  \
1     39.998596  116.339272  2009-02-14  04:52:35 2009-02-14 04:52:00   
2     39.998653  116.339214  2009-02-14  04:52:40 2009-02-14 04:52:00   
3     39.998641  116.339080  2009-02-14  04:52:45 2009-02-14 04:52:00   
4     39.998646  116.338932  2009-02-14  04:52:50 2009-02-14 04:52:00   
...         ...         ...         ...       ...                 ...   
8462  39.998501  116.338482  2009-02-14  23:39:35 2009-02-14 23:39:00   
8463  39.998525  116.338427  2009-02-14  23:39:40 2009-02-14 23:39:00   
8464  39.998501  116.338401  2009-02-14  23:39:45 2009-02-14 23:39:00   
8465  39.998461  116.338429  2009-02-14  23:39:50 2009-02-14 23:39:00   
8466  39.998337  116.338497  2009-02-14  23:39:55 2009-02-14 23:39:00   

     hour_block day_of_week  
0        f04t08    Saturday  
1        f04t08    Saturday  
2        f04t08    Saturday  
3  

In [44]:
file_name = r'C:\Users\sk002\OneDrive\바탕 화면\학교\Yoodori\Geolife Trajectories 1.3\Data\003\Trajectory\20081202160051.txt'
df5 = fileReader(file_name)
data_df_5 = gmeansFunc(df5)

       latitude   longitude        date      time            datetime   
0     39.999869  116.326993  2008-12-02  16:00:51 2008-12-02 16:00:00  \
1     39.999874  116.327013  2008-12-02  16:00:56 2008-12-02 16:00:00   
2     39.999895  116.327039  2008-12-02  16:01:01 2008-12-02 16:01:00   
3     39.999892  116.326940  2008-12-02  16:01:06 2008-12-02 16:01:00   
4     39.999889  116.326845  2008-12-02  16:01:11 2008-12-02 16:01:00   
...         ...         ...         ...       ...                 ...   
8240  39.999808  116.327688  2008-12-03  11:33:40 2008-12-03 11:33:00   
8241  39.999844  116.327620  2008-12-03  11:33:45 2008-12-03 11:33:00   
8242  39.999906  116.327541  2008-12-03  11:33:50 2008-12-03 11:33:00   
8243  40.000001  116.327446  2008-12-03  11:33:55 2008-12-03 11:33:00   
8244  40.000083  116.327361  2008-12-03  11:34:00 2008-12-03 11:34:00   

     hour_block day_of_week  
0        f16t20     Tuesday  
1        f16t20     Tuesday  
2        f16t20     Tuesday  
3  

In [45]:
data_df_1

,centers,clusters
0,"[39.9097909090909, 116.40469272727273]","[1037, 1038, 1039, 1040, 1041, 1042, 1043, 104..."
1,"[39.89846146153847, 116.38943146153848]","[1075, 1076, 1077, 1078, 1079, 1080, 1081, 108..."
2,"[40.00968718181818, 116.315785]","[211, 304, 333, 338, 477, 478, 479, 487, 489, ..."
3,"[40.009686225806455, 116.31517661290323]","[244, 249, 316, 319, 325, 327, 329, 422, 423, ..."
4,"[40.00974527586208, 116.31520817241378]","[318, 337, 350, 351, 352, 353, 354, 355, 356, ..."
5,"[40.009787730769226, 116.3152536923077]","[44, 326, 358, 362, 494, 535, 586, 587, 588, 6..."
6,"[40.00986117647058, 116.31530305882353]","[46, 313, 314, 331, 336, 343, 344, 364, 365, 3..."
7,"[40.00967733333333, 116.31529999999998]","[42, 217, 233, 243, 254, 328, 339, 349, 357, 3..."
8,"[40.00979785714286, 116.31541178571426]","[332, 345, 346, 347, 359, 360, 363, 374, 476, ..."
9,"[40.00954138888889, 116.31517327777777]","[227, 230, 246, 340, 390, 411, 412, 419, 421, ..."


In [11]:
data_df_2

,centers,clusters
0,"[39.992281250000005, 116.33119341666666]","[97, 98, 99, 100, 101, 102, 103, 104, 105, 106..."
1,"[39.9953895, 116.3278589]","[87, 88, 89, 90, 91, 92, 93, 94, 95, 96]"
2,"[40.00789702777777, 116.31948055555556]","[10, 11, 22, 23, 26, 27, 28, 29, 32, 33, 34, 3..."
3,"[39.90532633333333, 116.42230466666668]","[123, 124, 125, 126, 127, 128, 129, 130, 131, ..."
4,"[39.901377761904754, 116.4237179047619]","[138, 139, 140, 141, 142, 143, 144, 145, 146, ..."
5,"[36.34660016666667, 116.93652733333334]","[377, 378, 379, 380, 381, 382, 383, 384, 385, ..."
6,"[36.634264642857154, 116.84673960714284]","[349, 350, 351, 352, 353, 354, 355, 356, 357, ..."
7,"[36.172782416666664, 117.08250625000004]","[389, 390, 391, 392, 393, 394, 395, 396, 397, ..."
8,"[37.34021741666667, 116.34013833333334]","[307, 308, 309, 310, 311, 312, 313, 314, 315, ..."
9,"[36.95466855555554, 116.61113916666669]","[331, 332, 333, 334, 335, 336, 337, 338, 339, ..."


In [12]:
data_df_3

,centers,clusters
0,"[39.999050666666676, 116.33844224242424]","[338, 342, 348, 352, 355, 359, 361, 372, 373, ..."
1,"[39.9989764375, 116.33850056250002]","[336, 339, 351, 357, 375, 402, 414, 480, 529, ..."
2,"[39.99889454545455, 116.33841213636362]","[341, 345, 363, 378, 385, 388, 396, 481, 504, ..."
3,"[39.999181799999995, 116.33848291999999]","[337, 346, 347, 360, 370, 384, 387, 390, 391, ..."
4,"[39.99906608333333, 116.33864058333332]","[376, 484, 485, 494, 532, 533, 534, 537, 540, ..."
5,"[39.99913653846154, 116.33826815384614]","[447, 448, 457, 460, 463, 464, 468, 472, 473, ..."
6,"[39.99934622222221, 116.3385059047619]","[354, 358, 371, 383, 394, 399, 400, 404, 408, ..."
7,"[39.99965215384616, 116.33851211538462]","[403, 440, 445, 535, 554, 556, 557, 560, 565, ..."
8,"[39.99945103571429, 116.3384112857143]","[343, 349, 350, 353, 397, 398, 401, 417, 424, ..."
9,"[39.99875706060607, 116.3384714848485]","[340, 344, 356, 365, 367, 368, 369, 382, 386, ..."


In [13]:
data_df_4

,centers,clusters
0,"[39.99391791666667, 116.44537925000002]","[25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 3..."
1,"[39.992481454545455, 116.44658409090908]","[51, 52, 53, 54, 66, 67, 68, 69, 83, 84, 85]"
2,"[39.99105290000001, 116.4462241]","[56, 57, 58, 59, 60, 61, 62, 63, 64, 65]"
3,"[39.9920081, 116.333824]","[7, 8, 166, 167, 168, 169, 170, 171, 172, 173]"
4,"[39.991674999999994, 116.33188458333335]","[9, 10, 11, 12, 13, 14, 112, 113, 114, 163, 16..."
5,"[39.99170125, 116.32799696428573]","[120, 121, 122, 123, 124, 125, 126, 127, 137, ..."
6,"[39.99080900000001, 116.32797092857142]","[118, 119, 128, 129, 130, 131, 132, 133, 134, ..."
7,"[39.99903445000002, 116.33850331666665]","[193, 194, 212, 230, 280, 302, 304, 308, 315, ..."
8,"[39.998743555555556, 116.33858533333336]","[1, 188, 189, 191, 205, 209, 231, 267, 281, 30..."
9,"[39.99883130769231, 116.33870869230766]","[192, 199, 201, 214, 240, 241, 252, 266, 274, ..."


In [14]:
data_df_5

,centers,clusters
0,"[39.98266413043478, 116.3631911304348]","[564, 565, 566, 567, 568, 569, 570, 571, 572, ..."
1,"[40.011878749999994, 116.32156454545456]","[28, 40, 43, 46, 47, 48, 49, 50, 61, 63, 70, 7..."
2,"[40.01205111538461, 116.32167719230773]","[32, 33, 37, 38, 39, 41, 42, 51, 52, 58, 62, 6..."
3,"[40.01226710344827, 116.321580862069]","[27, 29, 30, 31, 34, 35, 36, 53, 54, 91, 92, 9..."
4,"[40.01232125, 116.32108841666667]","[60, 183, 230, 231, 232, 233, 234, 235, 236, 2..."
5,"[40.01186868965518, 116.32126286206895]","[59, 68, 185, 186, 187, 188, 191, 192, 193, 19..."
6,"[40.01154958620689, 116.321395]","[122, 125, 147, 149, 150, 151, 208, 214, 262, ..."
7,"[40.01158456250001, 116.3212083125]","[98, 99, 148, 184, 211, 212, 215, 219, 220, 22..."
8,"[40.01167257142857, 116.32152771428572]","[66, 96, 123, 124, 136, 146, 157, 158, 159, 26..."
9,"[40.01139318181817, 116.32169872727272]","[80, 100, 103, 113, 115, 116, 296, 297, 298, 2..."


In [15]:
print(f'df1 : {len(df1)} | df2 : {len(df2)} | df3 : {len(df3)} | df4 : {len(df4)} | df5 : {len(df5)}')

df1 : 1171 | df2 : 944 | df3 : 716 | df4 : 760 | df5 : 685


In [16]:
df

NameError: name 'df' is not defined

In [ ]:
df.iloc[1037]

In [17]:
data_df.clusters.iloc[0][0]

NameError: name 'data_df' is not defined

In [ ]:
data_df

In [ ]:
df['latitude'].iloc[1038]

In [ ]:
df['latitude'].iloc[0]

In [18]:
df1['labels'] = -1
for cluster_idx, (cluster, center) in enumerate(zip(data_df_1.clusters, data_df_1.centers)):
    for point_idx in cluster:
        df1['labels'].iloc[point_idx] = cluster_idx
        
df1

C:\Users\sk002\AppData\Local\Temp\ipykernel_10124\2373288560.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['labels'].iloc[point_idx] = cluster_idx
C:\Users\sk002\AppData\Local\Temp\ipykernel_10124\2373288560.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['labels'].iloc[point_idx] = cluster_idx
C:\Users\sk002\AppData\Local\Temp\ipykernel_10124\2373288560.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['labels'

,latitude,longitude,datetime,labels
0,40.001520,116.311859,2009-04-03 01:16:00,-1
1,40.001437,116.312250,2009-04-03 01:17:00,-1
2,40.002051,116.312630,2009-04-03 01:18:00,-1
3,40.004016,116.312476,2009-04-03 01:19:00,-1
4,40.004578,116.314774,2009-04-03 01:20:00,-1
...,...,...,...,...
1166,40.009642,116.315100,2009-04-04 14:31:00,9
1167,40.009683,116.315028,2009-04-04 14:32:00,7
1168,40.009551,116.315061,2009-04-04 14:33:00,10
1169,40.009591,116.315148,2009-04-04 14:34:00,9


In [19]:
df2['labels'] = -1
for cluster_idx, (cluster, center) in enumerate(zip(data_df_2.clusters, data_df_2.centers)):
    for point_idx in cluster:
        df2['labels'].iloc[point_idx] = cluster_idx
        
df2

C:\Users\sk002\AppData\Local\Temp\ipykernel_10124\1978855526.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['labels'].iloc[point_idx] = cluster_idx
C:\Users\sk002\AppData\Local\Temp\ipykernel_10124\1978855526.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['labels'].iloc[point_idx] = cluster_idx
C:\Users\sk002\AppData\Local\Temp\ipykernel_10124\1978855526.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['labels'

,latitude,longitude,datetime,labels
0,39.999840,116.327191,2009-01-20 00:28:00,-1
1,39.999991,116.326629,2009-01-20 00:29:00,-1
2,40.001186,116.325686,2009-01-20 00:30:00,-1
3,40.002164,116.324517,2009-01-20 00:31:00,-1
4,40.003486,116.323199,2009-01-20 00:32:00,-1
...,...,...,...,...
939,32.057004,121.270890,2009-01-21 02:11:00,-1
940,32.057173,121.269710,2009-01-21 02:16:00,-1
941,32.056679,121.271410,2009-01-21 02:17:00,-1
942,32.056826,121.271292,2009-01-21 02:18:00,-1


In [20]:
df3['labels'] = -1
for cluster_idx, (cluster, center) in enumerate(zip(data_df_3.clusters, data_df_3.centers)):
    for point_idx in cluster:
        df3['labels'].iloc[point_idx] = cluster_idx
        
df3

C:\Users\sk002\AppData\Local\Temp\ipykernel_10124\3144256473.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['labels'].iloc[point_idx] = cluster_idx
C:\Users\sk002\AppData\Local\Temp\ipykernel_10124\3144256473.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['labels'].iloc[point_idx] = cluster_idx
C:\Users\sk002\AppData\Local\Temp\ipykernel_10124\3144256473.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['labels'

,latitude,longitude,datetime,labels
0,39.999168,116.334129,2009-02-21 03:48:00,-1
1,39.999380,116.334245,2009-02-21 03:49:00,-1
2,40.000569,116.333664,2009-02-21 03:50:00,-1
3,40.000496,116.333412,2009-02-21 03:51:00,-1
4,40.000529,116.332941,2009-02-21 03:52:00,-1
...,...,...,...,...
711,39.996333,116.338616,2009-02-21 22:52:00,-1
712,39.996465,116.338802,2009-02-21 22:53:00,-1
713,39.996531,116.338755,2009-02-21 22:54:00,-1
714,39.997935,116.338913,2009-02-21 22:55:00,-1


In [21]:
df4['labels'] = -1
for cluster_idx, (cluster, center) in enumerate(zip(data_df_4.clusters, data_df_4.centers)):
    for point_idx in cluster:
        df4['labels'].iloc[point_idx] = cluster_idx
        
df4

C:\Users\sk002\AppData\Local\Temp\ipykernel_10124\3299083749.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df4['labels'].iloc[point_idx] = cluster_idx
C:\Users\sk002\AppData\Local\Temp\ipykernel_10124\3299083749.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df4['labels'].iloc[point_idx] = cluster_idx
C:\Users\sk002\AppData\Local\Temp\ipykernel_10124\3299083749.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df4['labels'

,latitude,longitude,datetime,labels
0,39.998569,116.339200,2009-02-14 04:52:00,16
1,39.998634,116.338622,2009-02-14 04:53:00,8
2,39.998102,116.337258,2009-02-14 04:54:00,-1
3,39.997688,116.334919,2009-02-14 04:55:00,-1
4,39.997469,116.333972,2009-02-14 04:56:00,-1
...,...,...,...,...
755,39.999533,116.338270,2009-02-14 23:35:00,25
756,39.998664,116.338281,2009-02-14 23:36:00,11
757,39.998998,116.337676,2009-02-14 23:37:00,-1
758,39.999212,116.338239,2009-02-14 23:38:00,22


In [22]:
df5['labels'] = -1
for cluster_idx, (cluster, center) in enumerate(zip(data_df_5.clusters, data_df_5.centers)):
    for point_idx in cluster:
        df5['labels'].iloc[point_idx] = cluster_idx
        
df5

C:\Users\sk002\AppData\Local\Temp\ipykernel_10124\2412946458.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df5['labels'].iloc[point_idx] = cluster_idx
C:\Users\sk002\AppData\Local\Temp\ipykernel_10124\2412946458.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df5['labels'].iloc[point_idx] = cluster_idx
C:\Users\sk002\AppData\Local\Temp\ipykernel_10124\2412946458.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df5['labels'

,latitude,longitude,datetime,labels
0,39.999869,116.326993,2008-12-02 16:00:00,14
1,39.999895,116.327039,2008-12-02 16:01:00,14
2,39.999880,116.325793,2008-12-02 16:02:00,14
3,40.000132,116.324713,2008-12-02 16:03:00,-1
4,40.001176,116.324511,2008-12-02 16:04:00,-1
...,...,...,...,...
680,39.996573,116.326611,2008-12-03 11:30:00,-1
681,39.998787,116.326686,2008-12-03 11:31:00,14
682,39.999864,116.327365,2008-12-03 11:32:00,14
683,39.999924,116.327458,2008-12-03 11:33:00,14


In [23]:
for i in range(-1, max(df1['labels']) + 1):
    print(len(df1[df1['labels'] == i]))


334
20
31
28
42
22
26
24
39
11
34
22
14
25
18
25
12
13
25
26
22
21
19
13
41
12
27
12
13
14
10
13
34
25
15
10
11
13
20
15
10
10


<h2>클러스터링 2번<h2>

In [ ]:
import folium

# 예제 데이터 생성 (서울의 위도, 경도)
locations = data_df_1.centers.values.tolist()

# 지도의 중심 좌표 설정
map_center = [sum(x[0] for x in locations) / len(locations), sum(x[1] for x in locations) / len(locations)]

# Folium 맵 생성
mymap = folium.Map(location=map_center, zoom_start=12)

# 위치 데이터를 지도에 마커로 표시
for loc in locations:
    folium.Marker(location=loc, popup='Location').add_to(mymap)

# 지도를 HTML 파일로 저장하거나 주피터 노트북에서 바로 표시
mymap


In [ ]:
# 예제 데이터 생성 (서울의 위도, 경도)
locations = data_df_2.centers.values.tolist()

# 지도의 중심 좌표 설정
map_center = [sum(x[0] for x in locations) / len(locations), sum(x[1] for x in locations) / len(locations)]

# Folium 맵 생성
mymap = folium.Map(location=map_center, zoom_start=12)

# 위치 데이터를 지도에 마커로 표시
for loc in locations:
    folium.Marker(location=loc, popup='Location').add_to(mymap)

# 지도를 HTML 파일로 저장하거나 주피터 노트북에서 바로 표시
mymap

In [ ]:
# 예제 데이터 생성 (서울의 위도, 경도)
locations = data_df_3.centers.values.tolist()

# 지도의 중심 좌표 설정
map_center = [sum(x[0] for x in locations) / len(locations), sum(x[1] for x in locations) / len(locations)]

# Folium 맵 생성
mymap = folium.Map(location=map_center, zoom_start=12)

# 위치 데이터를 지도에 마커로 표시
for loc in locations:
    folium.Marker(location=loc, popup='Location').add_to(mymap)

# 지도를 HTML 파일로 저장하거나 주피터 노트북에서 바로 표시
mymap

In [ ]:
# 예제 데이터 생성 (서울의 위도, 경도)
locations = data_df_4.centers.values.tolist()

# 지도의 중심 좌표 설정
map_center = [sum(x[0] for x in locations) / len(locations), sum(x[1] for x in locations) / len(locations)]

# Folium 맵 생성
mymap = folium.Map(location=map_center, zoom_start=12)

# 위치 데이터를 지도에 마커로 표시
for loc in locations:
    folium.Marker(location=loc, popup='Location').add_to(mymap)

# 지도를 HTML 파일로 저장하거나 주피터 노트북에서 바로 표시
mymap

In [ ]:
# 예제 데이터 생성 (서울의 위도, 경도)
locations = data_df_5.centers.values.tolist()

# 지도의 중심 좌표 설정
map_center = [sum(x[0] for x in locations) / len(locations), sum(x[1] for x in locations) / len(locations)]

# Folium 맵 생성
mymap = folium.Map(location=map_center, zoom_start=12)

# 위치 데이터를 지도에 마커로 표시
for loc in locations:
    folium.Marker(location=loc, popup='Location').add_to(mymap)

# 지도를 HTML 파일로 저장하거나 주피터 노트북에서 바로 표시
mymap

<h2>실루엣 계수<h2>

In [ ]:
df.iloc[1038]

In [ ]:
silhouette_avg = silhouette_score(df1[['latitude', 'longitude']], df1['labels'])
print(f'Silhouette Score: {silhouette_avg}')

silhouette_avg = silhouette_score(df2[['latitude', 'longitude']], df2['labels'])
print(f'Silhouette Score: {silhouette_avg}')

silhouette_avg = silhouette_score(df3[['latitude', 'longitude']], df3['labels'])
print(f'Silhouette Score: {silhouette_avg}')

silhouette_avg = silhouette_score(df4[['latitude', 'longitude']], df4['labels'])
print(f'Silhouette Score: {silhouette_avg}')

silhouette_avg = silhouette_score(df5[['latitude', 'longitude']], df5['labels'])
print(f'Silhouette Score: {silhouette_avg}')

<h2>T-SNE 시각화<h2>

In [ ]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import seaborn as sns

# 데이터 X와 레이블 y를 사용한다고 가정
tsne = TSNE(n_components=2, random_state=42)
X_tsne = tsne.fit_transform(df1[['latitude', 'longitude']])

sns.set_palette("deep")
sns.scatterplot(x = X_tsne[:, 0], y = X_tsne[:, 1], hue=df1['labels'])
plt.legend(title='Label')
plt.show()


In [ ]:
# 데이터 X와 레이블 y를 사용한다고 가정
tsne = TSNE(n_components=2, random_state=42)
X_tsne = tsne.fit_transform(df2[['latitude', 'longitude']])

sns.set_palette("deep")
sns.scatterplot(x = X_tsne[:, 0], y = X_tsne[:, 1], hue=df2['labels'])
plt.legend(title='Label')
plt.show()

In [ ]:
# 데이터 X와 레이블 y를 사용한다고 가정
tsne = TSNE(n_components=2, random_state=42)
X_tsne = tsne.fit_transform(df3[['latitude', 'longitude']])

sns.set_palette("deep")
sns.scatterplot(x = X_tsne[:, 0], y = X_tsne[:, 1], hue=df3['labels'])
plt.legend(title='Label')
plt.show()

In [ ]:
# 데이터 X와 레이블 y를 사용한다고 가정
tsne = TSNE(n_components=2, random_state=42)
X_tsne = tsne.fit_transform(df4[['latitude', 'longitude']])

sns.set_palette("deep")
sns.scatterplot(x = X_tsne[:, 0], y = X_tsne[:, 1], hue=df4['labels'])
plt.legend(title='Label')
plt.show()

In [ ]:
# 데이터 X와 레이블 y를 사용한다고 가정
tsne = TSNE(n_components=2, random_state=42)
X_tsne = tsne.fit_transform(df5[['latitude', 'longitude']])

sns.set_palette("deep")
sns.scatterplot(x = X_tsne[:, 0], y = X_tsne[:, 1], hue=df5['labels'])
plt.legend(title='Label')
plt.show()